In [1]:
import numpy as np

data = []
for i in range(32):
    for j in range(32):
        r=0
        g=i*j%255
        b=0
        a=255
        data = data + [r, g, b, a]
#data = np.array(data)        

In [2]:
len(data)

4096

In [3]:
import json
from IPython.display import Javascript

javascript = 'window.data={};'.format(json.dumps(data))
Javascript(javascript)


<IPython.core.display.Javascript object>

In [4]:
%%javascript
console.log(window.data.length)

<IPython.core.display.Javascript object>

create html element to render to

In [5]:
%%javascript
element.append("<div id='content'></div>");

<IPython.core.display.Javascript object>

Import js dependencies and boilerplate three.js code

In [6]:
%%javascript
requirejs.config({
    paths: { 
        'T' : ['//cdnjs.cloudflare.com/ajax/libs/three.js/r83/three'],                                   
    },
});
require(['T'], function(THREE) {
    console.log(THREE);
    var camera, scene, light, renderer;
    var geometry, material, mesh;
    var width = 600;
    var height = 400;
    var parentDiv = document.getElementById("content");
    
    init();
    animate();
    
    function init(){
        scene = new THREE.Scene();
        light = new THREE.AmbientLight(0xffffff, 0.8); // soft white light
          scene.add(light);
        camera = new THREE.PerspectiveCamera( 75, width / height, 1, 1000 );
        camera.position.z = 500;
        var ta = Uint8Array.from(window.data);
        console.log(ta)
        var texture = new THREE.DataTexture(ta, 32, 32, THREE.RGBAFormat  )
        texture.needsUpdate = true;
        geometry = new THREE.SphereGeometry(200, 64, 64);
        material =  new THREE.MeshLambertMaterial({
            map: texture 
        })
        mesh = new THREE.Mesh(geometry, material);
        scene.add( mesh );
  
        renderer = new THREE.WebGLRenderer({antialias: true});
        renderer.setSize(width, height);
        renderer.setClearColor('white');
        while (parentDiv.firstChild) {
            parentDiv.removeChild(parentDiv.firstChild);
        }
        parentDiv.appendChild( renderer.domElement ) ;
    }  
    
    function animate(){
        window.requestAnimationFrame( animate );
        mesh.rotation.x = Date.now() * 0.00005;
        mesh.rotation.y = Date.now() * 0.0001;  
        renderer.render( scene, camera);
    }
    
});

<IPython.core.display.Javascript object>